In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_machine_learning
!pip install qiskit_algorithms
!pip install matplotlib
!pip install pylatexenc
!pip install scipy
!pip install -U scikit-learn

!git clone https://github.com/IsaVia777/atelier_qml.git

# Lab 3: Classification

**Objectives**
- Classify data in higher dimensional space using 1 qubit 
- Measurement along $X$,$Y$,$Z$ axis  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import QuantumCircuit
from qiskit_aer import Aer

SEED = 8398

In [ ]:
import sys
sys.path.insert(0, '/content/atelier_qml')

from utils import *

In [ ]:
qasm_sim = Aer.get_backend('qasm_simulator')
sv_sim = Aer.get_backend('statevector_simulator')

### 1D linearly separable dataset

In [ ]:
x,y,x0,x1 = get_seperable_data()
nb_features = 1

# Let's see what our dataset look like. Label 0 in blue, label 1 in red
plt.yticks([])
plt.scatter(x0, [0]*len(x0), color='blue')
plt.scatter(x1, [0]*len(x1), color='red')

Embedding datapoints in the rotation angle about the X-axis

In [ ]:
def angle_embedding(x_params,nb_features):
       
    """
    Qubit - or rotation - encoding in RX gates.

    :param x_params: List of parameters to embed.
    :param nb_features: The number of features of the feature vector. 
    :return: The quantum circuit with the embedding layer. 
    """
    
    qc = QuantumCircuit(nb_features)

    for i in range(nb_features):
        qc.rx(x_params[i], i)

    return qc 


In [ ]:
# Create a list of `Parameter`. Since the data points have only one feature
# there will be only one parameter in this list.
x_params = [Parameter(f'x{str(i)}') for i in range(nb_features)]

# Create the parametrized embedding layer
qc = angle_embedding(x_params, nb_features)

qc.draw('mpl')

In [ ]:
emb_circuit = qc.assign_parameters(x[0])

print(f'Encoding for datapoint {x[0]}:')
emb_circuit.draw('mpl')

### Using angle embedding to map this data into a quantum feature space. 

In [ ]:
statevectors0 = get_statevector(qc, x0, x_params, sv_sim)
statevectors1 = get_statevector(qc, x1, x_params, sv_sim)
plot_bloch_visualization([statevectors0, statevectors1], ['b', 'r'])

## Implementation of the classification algorithm
![lab3.png](lab3.png)

In [ ]:
shots = 1024


def get_measurement_outcomes(data, emb_circuit, rot_circuit=None):
    """
    Run the circuit of the binary quantum classifier for all the data vectors of a dataset.

    :param data:        Dataset to classify
    :param emb_circuit: Angle embedding circuit.
    :param rot_circuit: Circuit defining the measurement basis.
                        Defaults to the computational basis (Z)

    :return: A list of measurement results, one per data vector in the dataset.
    """
    # Add the circuit defining the measurement basis.
    if rot_circuit != None:
        emb_circuit = emb_circuit.compose(rot_circuit)

    circuits = []
    # For each datapoint in the dataset
    for x_i in data:
        x_params = emb_circuit.parameters
        # Dictionary associating one data feature to one parameter in the circuit
        x_params_dict = {p:v for (p,v) in zip(x_params, x_i)}
        # Replace the circuit parameters by their associated values
        gqc = emb_circuit.assign_parameters(x_params_dict)
        # Ajouter la mesure de tous les qubits
        # Add the measurement instruction for all qubits
        gqc.measure_all()
        circuits.append(gqc)

    # Run the circuit "N" times (determined by the number of shots)
    result = qasm_sim.run(circuits, shots=shots).result()
    counts = result.get_counts()

    return counts

In [ ]:
counts = get_measurement_outcomes(x, qc)
counts

In [ ]:
def get_probabilities_from_counts(counts):
    """
    Compute the probabilities of belonging to class 0 or 1 according to 
    the measurement results of a binary classifier.
    """
    num_classes = 2
    probs = []

    # For each measurement results corresponding to a datapoint in the dataset
    for dict_meas in counts:
        # We count the number of times we observed 0 and 1 respectively
        indiv_probs = np.zeros(num_classes)
        indiv_probs[0] = dict_meas.get('0', 0)
        indiv_probs[1] = dict_meas.get('1', 0)
        # We translate the number of observation instances of 0 and 1 into probabilities
        indiv_probs /= shots
        probs.append(indiv_probs)

    return np.round(probs, decimals=3)

In [ ]:
probs = get_probabilities_from_counts(counts)
probs

In [ ]:
def get_accuracy(probs, targets):
    """
    Compute the accuracy obtained by the binary classifier

    :param probs:   The probability predicted by the classifier for each data vector to be
                    in class 0 or 1.

    :param targets: List of labels.

    :return: classifier precision.
    """
    # The predicted class correponds to the one with the highest probability
    predict = np.argmax(probs, axis=1)
    targets = np.array(targets).reshape(predict.shape)
    # We count the number of times the prediction corresponds to the label and we divide
    # by the total number of predictions.
    accuracy = np.sum(predict == targets) / len(predict)
    
    return accuracy

In [ ]:
accuracy = get_accuracy(probs, y)
print(f'>\n> The accuracy of our classifier is {accuracy*100}%.\n>')

In [ ]:
predict = np.argmax(probs, axis=1)
plot_bloch_visualization([statevectors0, statevectors1], x=x, x0=x0, x1=x1, score=(predict==np.array(y).reshape(predict.shape)))

The X points are those who have been misclassified.<br>
Are you surprised?<br>
How could we do better?


## Measuring along the Y-axis

To measure along the Y-axis, we proceed by applying $|\psi'\rangle = HS^\dagger|\psi\rangle$ where the $S$ gate is a $\phi = \frac{\pi}{2}$ rotation around the Z-axis.

In [ ]:
qc_ybasis = QuantumCircuit(1)
qc_ybasis.sdg(0)
qc_ybasis.h(0)

qc_ybasis.draw('mpl')

In [ ]:
counts = get_measurement_outcomes(x, qc, qc_ybasis)
probs = get_probabilities_from_counts(counts)
accuracy = get_accuracy(probs, y)
print(f'>\n> The accuracy of our classifier is {accuracy*100}%.\n>')

In [ ]:
predict = np.argmax(probs, axis=1)
plot_bloch_visualization([statevectors0, statevectors1], x=x, x0=x0, x1=x1, score=(predict==np.array(y).reshape(predict.shape)))

# Exercise 3: 
 * Implement the angle embedding such that the rotations are performed about the Y-axis.
 * Implement the quantum circuit `qc_xbasis` which performs measurements in the X basis, by simply adding a $H$ gate.<br>
 Validate your results by making sure you obtain an accuracy of 100%!

In [ ]:
def angle_embedding_ry(x_params, nb_features):
    """
    Angle embedding using RY gates.

    :param x_params: List of parameters to embed.
    :param nb_features: Number of features in a data vector.

    :return: The quantum circuit applying the angle embedding.
    """

    ## Your code here!
    qc = None

    return qc 

In [ ]:
qc = angle_embedding_ry(x_params, 1)
statevectors0 = get_statevector(qc, x0, x_params, sv_sim)
statevectors1 = get_statevector(qc, x1, x_params, sv_sim)
plot_bloch_visualization([statevectors0, statevectors1], ['b', 'r'])

In [ ]:
## Your code here!

qc_xbasis = None

qc_xbasis.draw('mpl')

In [ ]:
## Your code here!

counts = None
probs = None
accuracy = None
print(f'>\n> The accuracy of our classifier is {accuracy*100}%.\n>')